# Hybrid Business Process Simulation
A proof of concept example for updating the DES model of a process based on the system dynamics model of the process.
- The DES and SD models are taken as input. 
- For the ease of use and performing experiments with your desired process, the following supports are provided:
-- To generate a CPN model of your process and the corresponding SML file using an event log, you can use the following link, in case of any specific scenario you need to change the process model in the CPN Tools. The link only generates a CPN model based on the given event log, further desired changes should be done by the user.
-- https://cpn-model-process-discovery-1.herokuapp.com/generate-cpn-model/
-- In case you want to generate your SD Logs and SD  models, this tool supports you 
https://github.com/mbafrani/PMSD
-- Note that you need to make sure that there are common variables names, e.g., service time, arrival rate, ..., are in both models that can be updated. 

#### To run the current example:
1. Run the CPN model (it should be a large period enough)
2. Run the main method in the script (it will automatically update the CPN model while running)
3. The results (before and after updates based on SD results) are captured in one event log continuously. 

In [ ]:
#install pysd for executing system dynamics library in python
!pip install pysd

In [ ]:

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pysd
from collections import defaultdict
import seaborn as sns
from scipy.stats import variation, ks_2samp, shapiro

class SimVal:

    def read_model(self, sd_model,sd_log_file):
        direct_var = []
        stock_var =[]
        model = pysd.read_vensim(sd_model)
        with open(os.path.join(sd_model), 'r') as f:
            body = f.readlines()
            for b in body:
                if 'A FUNCTION' in b:
                    varname=b.split('=')[0].replace("  ","")
                    direct_var.append(varname)
        x = dir(model.components)

        for i in x:
            if 'integ' in i:
                stock_var.append(i[7:])

        variable_involeved_val = model.run()
        variable_involeved = variable_involeved_val.columns.str.replace(' ', '_')
        df_real_val = pd.read_csv(sd_log_file)

        var_dict_params = {}
        df_real_val.columns = df_real_val.columns.str.replace(' ', '')
        for name in variable_involeved:
            #if name in df_real_val.columns and name.lower() not in stock_var:

            if name in df_real_val.columns and name in direct_var:
                v = df_real_val[name]
                # v = v[:12]
                vname = name.lower()

                var_dict_params[vname]= v

        stocks = model.run(params=var_dict_params)
        sim_values = stocks

        sim_values = sim_values.where(sim_values >0, other=0)
        variables_names = stocks.columns
        return sim_values

    def creat_real_sim_dict(self, sd_log, sim_values):
        real_sim_dict = defaultdict(list)
        real_values_sd_log=pd.read_csv(sd_log)

        #sim_values.columns = sim_values.columns.str.replace(' ', '_')
        sim_values.columns = sim_values.columns.str.replace('_', '')
        sim_values.columns = sim_values.columns.str.replace(' ', '')
        real_values_sd_log.columns = real_values_sd_log.columns.str.replace(' ', '')
        real_val_names = real_values_sd_log.columns

        for rname in real_val_names:
            rname.replace("_", "")
            rname.replace("_", "")


            if rname in sim_values.columns:
                sim_size = len(sim_values[rname])
                real_siz = len(real_values_sd_log[rname])
                if sim_size <= real_siz:
                    real_list = real_values_sd_log[rname]
                    real_sim_dict[rname].append(real_list[:sim_size])
                    real_sim_dict[rname].append(sim_values[rname])
                else:
                    sim_list = sim_values[rname]
                    real_sim_dict[rname].append(real_values_sd_log[rname])
                    real_sim_dict[rname].append(sim_list[:real_siz])

        return real_sim_dict

    def validate_results(self,real_sim_dict):

            plt.rcParams.update({'font.size': 10})
            val_image_names =[]
            plt.tight_layout()
            for vname, vvalue in real_sim_dict.items():
                plt.figure()
                simulation_list = []
                simulation_list = vvalue[1].values
                real_list = []
                real_list = vvalue[0]

                if len(simulation_list) != len(real_list):
                   #real_list = real_list[:(len(simulation_list)-len(real_list))].values
                    simulation_list=simulation_list[:-1]
                # data description
                plt.subplot(3, 3, 1)
                plt.tight_layout()
                plt.gca().set_title("reality vs simulation",size= 8)
                data = [real_list, simulation_list]
                #plt.plot(np.cumsum(real_list))
                plt.plot(real_list,)
                plt.plot(simulation_list,'--')

                plt.subplot(3, 3, 4)
                plt.tight_layout()
                plt.gca().set_title("distribution of reality vs simulation",size = 8)
                sns.distplot(real_list,color="blue", hist=False)
                sns.distplot(simulation_list,color="orange", kde_kws={'linestyle':'--'})
                statistical_compare = pd.DataFrame(
                    [[np.mean(real_list), np.std(real_list), variation(real_list)],
                     [np.mean(simulation_list), np.std(simulation_list),variation(simulation_list)]],
                      index=['reality', 'simulated'],columns=['mean', 'STD', 'CV'])

                plt.subplot(3, 3,(2,3) )
                plt.tight_layout()
                plt.axis('off')
                plt.text(0.1, 0.2, "reality:"+
                         str(statistical_compare.loc['reality',:])+"\n simulation:"+
                         str(statistical_compare.loc['simulated', :]), fontsize=9)

                # Test Distribution of Data
                sta, p_value = ks_2samp(real_list, simulation_list)
                plt.subplot(3, 3, (5,6))
                plt.tight_layout()

                plt.axis('off')
                if p_value >= 0.05:
                    plt.text(0.1, 0.6, "P value of the Kolmogrov Test:\n" + str(round(p_value,2))+ " (Two distributions are similar)", fontsize=9)
                else :
                    plt.text(0.1, 0.6,"P value of the Kolmogrov Test:\n" + str(round(p_value,2)) + " (Two distributions are not similar)",fontsize=9)

                # Test PairWise
                diff_list = []
                diff_list = np.array(diff_list)
                diff_list = abs(np.array(real_list) - np.array(simulation_list))
                plt.subplot(3, 3,7)
                plt.tight_layout()
                plt.gca().set_title("Distribution of Difference",size = 9)
                if np.sum(real_list) != 0:
                    sns.distplot(diff_list, color='green', rug=True, hist=False)

                plt.subplot(3, 3, (8,9))
                plt.tight_layout()
                plt.axis('off')
                acceptance_rate = (sum(i <= 0.20 for i in diff_list) / len(real_list)) * 100
                # diff_normality_test
                if len(diff_list) >2:
                    p, stat = shapiro(diff_list)
                    if p > 0.5:
                        plt.text(0.1, 0.3, "Differenc follows a normal distribution \n"
                                           " with coefficient of varaition of: " + str(round(variation(diff_list),2)))
                    else:
                        plt.text(0.1, 0.3, "Differenc is not a normal distribution \n"
                                           " mean and standard deviation of: " + str(round(variation(diff_list),2)))

                if len(np.nonzero(real_list)) >0 and len(np.nonzero(simulation_list))>0:
                    normal_real_list = (real_list - np.min(real_list))/(np.max(real_list)-np.min(real_list))
                    normal_simulation_list = (simulation_list - np.min(simulation_list)) / (np.max(simulation_list) - np.min(simulation_list))
                    normal_diff_list = abs(np.array(normal_real_list) - np.array(normal_simulation_list))
                val_image_names.append(vname)
                plt.subplots_adjust(wspace=0,hspace=1)
                plt.savefig('static/images/'+str(vname)+'_validation.png')

            return val_image_names


In [ ]:

import pandas as pd
import pysd
#from SimulationSD import SimVal

class SD2DES:

    def update_sml(self, cpn_address, attributes):
        with open(cpn_address,"r")as file:
            lines= file.readlines()
            for line in lines:
                for kv, vv in attributes.items():
                    if "val "+str(kv) + "=" in line:
                        filedata = open(cpn_address, 'r')
                        filedata = filedata.read()
                        filedata = filedata.replace(line, "val " + str(kv) + " = " + str(round(vv.values[0])) + ";\n")
                        f = open("Changed_cpn_model.sml", 'w')
                        f.write(filedata)
                        f.close()
        return

    def run_simulation(self,sdmodel,sd_log):
        vensimmodel = pysd.read_vensim(sdmodel)
        sd_result = vensimmodel.run()
        return sd_result

    def read_sd_result(self,sd_address):
        sd_result = pd.read_csv(sd_address)
        sd_result_dict= []
        for col in sd_result.columns:
            sd_result_dict[col]=sd_result[col][-1]
        return sd_result_dict


**Recommended**:
#### Using the simulated SD-Log 
- Design your scenario in the SD model (SFD) and execute it in SD software such as Vensim
- Store the simulation results as a table in .csv.(see the example SDLog.csv)
- And feed it to update the CPN model. 
- If you want to run the SD model (SFD) directly in this code, keep in mind that it should not include the distribution function because the pysd library cannot handle it. 

In [ ]:

#Providing Inputs:
    
#SD Log Address (.csv)
sd_log_file = r"SDLog.csv"
updated_sd_log_file = r"Updated_SDLog.csv"
# SML File Address including the CPN model functions (.sml)
sml_file = r"TWTest.sml"
# Current Event log Address (.csv)
event_log=pd.read_csv(r'TWTest8-17-5days.csv')

#Execute the framework:

sd2des= SD2DES() 
#Update sml file (cpn model)
sim = SimVal()
#Read SD Log, initiate SD model
sim_values= pd.read_csv("Updated_SDLog.csv")
real_sim_dict= sim.creat_real_sim_dict(sd_log_file, sim_values)
sd2des.update_sml(sml_file, sim_values.tail(1)) #Update SML file using the last value of simulate SD log 
#read the newly generated Event log
new_event_log=pd.read_csv(r'TWTest8-17-5days.csv')
print(new_event_log)


#### Using the SD simulation engine directly in the framework 
- If your SFD model  includes any distribution or sophisticated functions, utilize the above configuration. 

In [ ]:
#SD Log Address (.csv)
sd_log_file = r"SDLog.csv"
#SD Model Address (.mdl)
sd_model = r'CallCenterTestResourceEfficiency.mdl' 
#SML File Address including the CPN model functions (.sml)
sml_file = r"TWTest.sml"
#Current Event log Address (.csv)
event_log=pd.read_csv(r'TWTest8-17-5days.csv')


#Execute SD model, Update SD Log, Update sml file (cpn model):

#Execute SD model
sd2des= SD2DES()
#Execute SD model, Update SD Log, Update sml file (cpn model)
sim = SimVal()
#Read SD Log, initiate SD model
sim_values= sim.read_model(sd_model,sd_log_file)
#Execute SD model and compare real and simulated values in SD logs
real_sim_dict= sim.creat_real_sim_dict(sd_log_file, sim_values)
#Update SML file using the last value of simulate SD log 
sd2des.update_sml(sml_file, sim_values.tail(1)) 
#read the newly generated Event log
new_event_log=pd.read_csv(r'TWTest8-17-5days.csv')

